In [25]:
from fastapi import FastAPI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from  langserve import add_routes



In [26]:
load_dotenv()

qroq_apikey = os.getenv("GROQ_API_KEY")

model = ChatGroq(model="llama-3.1-8b-instant", groq_api_key=qroq_apikey)

In [27]:
#
from langchain_core.messages import HumanMessage
human_mesge = [HumanMessage(content="Hi, my name is albin and i am cheif ai engineer")]
model.invoke(human_mesge)

AIMessage(content="Nice to meet you, Albin. As the Chief AI Engineer, I'm sure you have a fascinating job. What specific area of AI are you working in? Are you focused on developing new models, improving existing ones, or exploring the applications of AI in various industries?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 56, 'prompt_tokens': 49, 'total_tokens': 105, 'completion_time': 0.091822925, 'prompt_time': 0.002876904, 'queue_time': 0.08842265, 'total_time': 0.094699829}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_7b3cfae3af', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--fc0c43d9-530b-4f39-af08-a536ea7585d2-0', usage_metadata={'input_tokens': 49, 'output_tokens': 56, 'total_tokens': 105})

In [28]:
from langchain_core.messages import AIMessage

ai_msge = [HumanMessage(content="Hi, my name is albin and i am cheif ai engineer"),
           AIMessage(content="Nice to meet you, Albin. As a Chief Operations and Maintenance (O&M) Engineer, you play a crucial role in ensuring the smooth operation and reliability of facilities, equipment, and processes. Your expertise is essential in maintaining the efficiency and productivity of the organization.\n\nWhat specific challenges or projects are you currently working on, or would you like to discuss about O&M engineering in general? "),
           HumanMessage(content="Hey, what is my name ? what i am do ?"),
           ]


model.invoke(ai_msge)

AIMessage(content='Your name is Albin, and you are a Chief AI Engineer.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 150, 'total_tokens': 165, 'completion_time': 0.02297152, 'prompt_time': 0.00813878, 'queue_time': 0.08831208, 'total_time': 0.0311103}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_e32974efee', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--70768059-3ff5-482c-9bde-5b6da22c0c8f-0', usage_metadata={'input_tokens': 150, 'output_tokens': 15, 'total_tokens': 165})

# Message History

In [29]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [30]:
#if multoplie pepole chat with the model => how to hnadle it ==> session

store = {}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory() # craete new one if not found if
    return store[session_id] #retrun chat associated with that session id
        
with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [31]:
config1 = {"configurable" : {"session_id": "chat1"}}

In [32]:
responce = with_message_history.invoke(
    [HumanMessage(content="My name is albin, and i am learning GenAI")],
    config=config1
    )

responce.content

"Nice to meet you, Albin. I'm happy to help you learn about GenAI (Generative AI). GenAI is a rapidly evolving field that deals with generating new, original content, such as text, images, music, and more. It's an exciting area of research that has the potential to revolutionize various industries and aspects of our lives.\n\nTo get started, let's break down the basics of GenAI:\n\n1. **Types of GenAI**: There are several types of GenAI models, including:\n\t* **Text-to-Text**: These models generate text based on input text, such as language translation, text summarization, and chatbots.\n\t* **Image-to-Image**: These models generate images based on input images, such as image synthesis and image editing.\n\t* **Audio-to-Audio**: These models generate audio based on input audio, such as music generation and voice synthesis.\n2. **Key Concepts**: Understanding the following concepts is essential for learning GenAI:\n\t* **Neural Networks**: The building blocks of GenAI models, which are

In [ ]:
#change > sessionid

config1 = {"configurable" : {"session_id": "chat2"}}
responce = with_message_history.invoke(
    [HumanMessage(content="what is my name")],
    config=config1
    )

responce.content


"I don't have any information about your name as our conversation has just started. If you'd like to share it with me, I'd be happy to know."

In [ ]:
#check the chat message histroy => sessionid

config1 = {"configurable" : {"session_id": "chat1"}}
responce = with_message_history.invoke(
    [HumanMessage(content="what is my name")],
    config=config1
    )

responce.content

'Your name is Albin.'

# prompt template

In [35]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

In [38]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system" , "You are powerful ai model, Answer all question at your very best..."),
        MessagesPlaceholder(variable_name="messages")
        
    ]
)

chain = prompt | model
result = chain.invoke({"messages" : [HumanMessage(content="My name is Albin")]})
result.content

"Nice to meet you, Albin. I'm glad to be chatting with you. Is there something I can help you with, or would you like to just have a conversation?"

In [42]:
# chat history
with_msge_history = RunnableWithMessageHistory(chain, get_session_history)

config = {"configurable" : {"session_id" : "chat3"}}
res  = with_message_history.invoke(
        [HumanMessage(content="My name is Albin")],
        config=config
    )
res.content

'Nice to meet you, Albin. Is there something I can help you with today?'

In [46]:
## add more complexity
prompt = ChatPromptTemplate.from_messages(
    [
        ("system" , "You are powerful ai model, Answer all question at your very best in {language}"),
        MessagesPlaceholder(variable_name="messages"),
        
    ]
)

chain = prompt | model
result = chain.invoke(
    {
        "messages" : [HumanMessage(content="My name is Albin")], 
        "language" : "German"
    }
)
result.content

"Hallo Albin, ich bin froh, dich kennenzulernen! (Hello Albin, I'm happy to meet you!) Wie kann ich dir heute helfen? (How can I help you today?)"

In [50]:
# chat history
with_msge_history = RunnableWithMessageHistory(chain, get_session_history, input_messages_key="input")

config = {"configurable" : {"session_id" : "chat4"}}
res2  = with_message_history.invoke(
    {
        "input" : [HumanMessage(content="My name is Albin")], 
        "language" : "German"
    },
        config=config 
    )
res2.content

"It's nice to meet you, Albin again. How's your day going so far?"